In [1]:
# Import packages
import pandas as pd 
import numpy as np 
import geopandas as gpd 
import matplotlib.pyplot as plt

from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import io
import json

import plotly.express as px
from collections import Counter
import glob


## Open All CSV's - only do this if you have ~1.5 Gb RAM available, else your PC will close some apps without saving.

In [13]:

path = "../data/external/province/new" # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

100%|██████████| 101/101 [00:00<00:00, 144.46it/s]


In [14]:
# Changing variables to the correct dType

frame["humidity"] = pd.to_numeric(frame["humidity"])
frame["pm2_5"] = pd.to_numeric(frame["pm2_5"])
frame["pressure"] = pd.to_numeric(frame["pressure"])
frame["trip_sequence"] = pd.to_numeric(frame["trip_sequence"])
frame['recording_time'] = pd.to_datetime(frame['recording_time'], format="%Y-%m-%d %H:%M:%S")
frame["date"] = frame['recording_time'].dt.date
frame["year"] = frame['recording_time'].dt.year
frame["month"] = frame['recording_time'].dt.month


In [15]:
# Descriptive statistics, takes about 1-2 min.
frame.describe()

,lon,lat,trip_sequence,humidity,pm2_5,pressure,temperature,distance,delta_time,avg_speed_ms,year,month
count,19043.000000,19043.000000,19043.000000,19043.000000,19043.000000,19043.000000,19043.000000,19009.000000,19009.000000,19004.000000,19043.000000,19043.000000
mean,5.112887,52.088768,24.817466,55.233314,7.296644,1016.632043,25.841417,1723.212410,355.185912,4.351279,2020.142099,7.416636
std,0.178315,0.082545,30.615449,18.782282,7.708994,15.428247,164.138501,27969.604731,4101.317880,2.600528,0.349161,3.189433
min,4.863109,51.859978,0.000000,16.000000,1.000000,957.000000,0.700000,0.000000,0.000000,1.400612,2020.000000,1.000000
25%,4.987329,52.020802,4.000000,39.000000,2.000000,1006.000000,14.900000,4.481281,1.000000,2.227931,2020.000000,6.000000
50%,5.057683,52.083187,10.000000,56.000000,5.000000,1014.000000,21.500000,8.578482,2.000000,3.567520,2020.000000,6.000000
75%,5.207186,52.201355,32.000000,71.000000,9.000000,1026.000000,28.300000,15.878805,4.000000,5.845668,2020.000000,11.000000
max,5.584312,52.277798,156.000000,99.000000,146.000000,1080.000000,6553.300000,903063.666360,86386.000000,12.499906,2021.000000,12.000000


In [ ]:
frame.info()

In [26]:
frame.groupby(["year","month"]).count()


In [27]:
test2

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm2_5,pressure,temperature,distance,delta_time,avg_speed_ms,date
month,,,,,,,,,,,,,,
1,951977,951977,951977,951977,951977,951977,951977,951977,951977,951977,951972,951972,951972,951977
2,666956,666956,666956,666956,666956,666956,666956,666956,666956,666956,666952,666952,666952,666956
3,748107,748107,748107,748107,748107,748107,748107,748107,748107,748107,748103,748103,748103,748107
4,689330,689330,689330,689330,689330,689330,689330,689330,689330,689330,689325,689325,689325,689330
5,786708,786708,786708,786708,786708,786708,786708,786708,786708,786708,786704,786704,786704,786708
6,634677,634677,634677,634677,634677,634677,634677,634677,634677,634677,634673,634673,634673,634677
7,642243,642243,642243,642243,642243,642243,642243,642243,642243,642243,642238,642238,642238,642243
8,606401,606401,606401,606401,606401,606401,606401,606401,606401,606401,606396,606396,606396,606401
9,636637,636637,636637,636637,636637,636637,636637,636637,636637,636637,636633,636633,636633,636637


In [16]:
count = Counter([date for date in frame["date"]])

df = pd.DataFrame.from_dict(count, orient='index', columns=["activity"]).reset_index().sort_values(by=['index'])

fig = px.line(df, x='index', y='activity', title='Snuffelfiets activity per day')
fig.show()

In [25]:
url = 'https://ckan.dataplatform.nl/api/3/action/package_show?id=3660d2e1-84ee-46bf-a7b6-7e9ac1fcaf3a&include_tracking=true'
r = requests.get(url)
data = r.json()

csv_urls = [x['url'] for x in data['result']['resources']]
len((csv_urls))

109

In [37]:
files = [x[-25:] for x in csv_urls]
len(set(files))

101

In [38]:
url = 'https://ckan.dataplatform.nl/api/3/action/package_show?id=9cc4de28-6d03-4b59-8c66-085b3e8b3956&include_tracking=true'
r = requests.get(url)
data = r.json()

csv_urls = [x['url'] for x in data['result']['resources']]
len((csv_urls))

103

In [39]:
files = [x[-25:] for x in csv_urls]
len(set(files))

101

In [ ]:
mean_pm25_date = frame[["date", "pm2_5"]].groupby("date").mean()

fig = px.scatter(mean_pm25_date, x=mean_pm25_date.index, y='pm2_5', title='Mean PM2.5 concentration per day (ug/m3)')
fig.show()

In [ ]:
frame["week"] = frame["recording_time"].dt.isocalendar().week
frame["month"] = frame["recording_time"].dt.month

mean_speed_week = frame[["week", "avg_speed_ms"]].groupby("week").mean()

fig = px.scatter(mean_speed_week, x=mean_speed_week.index, y='avg_speed_ms', title='Mean avg. speed per week (m/s)')
fig.show()

In [ ]:
activity = Counter([sensor for sensor in frame["sensor"]])

df = pd.DataFrame.from_dict(activity, orient='index', columns=["activity"]).reset_index().sort_values(by=['activity'], ascending=False)
df.replace(to_replace=df['index'].unique(), value=range(0,len(activity)), inplace=True) # change sensor name to rank


fig = px.bar(df, x='index', y='activity', title='Most active users/sensors')
fig.show()

In [ ]:
df = frame[["date", "sensor", "pm2_5"]].groupby(['date', 'sensor']).count()
df.reset_index(inplace=True)
df.replace(to_replace=df['sensor'].unique(), value=range(0,len(df['sensor'].unique())), inplace=True) # change sensor names to a number.

fig = px.line(df, x='date', y='pm2_5', title='Daily activity per sensor', color='sensor')
fig.show()

In [ ]:
import seaborn as sns 

frame["humidity"] = pd.to_numeric(frame["humidity"])
frame["pm2_5"] = pd.to_numeric(frame["pm2_5"])
frame["pressure"] = pd.to_numeric(frame["pressure"])

correlation = frame[["lon", "lat", "humidity", "pm2_5", "pressure", "temperature", "distance", "avg_speed_ms"]].corr()
sns.heatmap(correlation, cmap="GnBu", annot=True)

In [ ]:
df = frame[['sensor', 'pm2_5', 'date']].groupby(["sensor", "date"]).mean()
df.reset_index(inplace=True)
df.replace(to_replace=df['sensor'].unique(), value=range(0,len(df['sensor'].unique())), inplace=True) # change sensor names to a number.
df.head()

In [ ]:
fig = px.line(df, x='date', y='pm2_5', title='PM2.5 daily averages per sensor', color='sensor')
fig.show()

In [ ]:
df[df["pm2_5"] > 100].sensor.unique()

In [ ]:
len(df[df["pm2_5"] > 100].sensor.unique())/len(df.sensor.unique())

In [ ]:
#fig = px.histogram(df, x='pm2_5', title='PM2.5 daily averages per sensor', color='sensor')
#fig.show()

### Possible steps:
- Exclude all sensors with <X observations/days
- Exclude all sensors that are an outlier in terms of total average PM2.5
- Exclude all sensors that are an outlier in terms of weekly average PM2.5

In [ ]:
df = frame[['sensor', 'pm2_5']].groupby(["sensor"]).mean()
df.reset_index(inplace=True)

px.box(df, y="pm2_5")

In [ ]:
df[df["pm2_5"] > 14].sensor.unique()

In [ ]:
df[df["pm2_5"] > 14]

In [ ]:
len(frame[frame["sensor"]=="bd6b87eceb4104d42d9366942c41d82d"])

In [ ]:
csv_file = "C:/Users/Klant/Documents/GitHub/ADS-Snuffelfiets-Thesis/data/external/resource_2021_03_29_2021_04_05.csv"

df = pd.read_csv(csv_file)
df.head()

In [ ]:
df["date"] = pd.to_datetime(df['recording_time'], format="%Y-%m-%d %H:%M:%S").dt.date
df['pm2_5_log'] = np.log(df["pm2_5"])


means = df.groupby(["sensor", 'date']).mean()
means.reset_index(inplace=True)


fig = px.histogram(means, x='pm2_5_log', title='PM2.5-log daily averages per sensor', color='sensor')
fig.show()

## Rasterizing Point Data - work in progress!
Source: https://corteva.github.io/geocube/stable/examples/rasterize_point_data.html

In [ ]:
import json
from functools import partial

import geopandas as gpd
from shapely.geometry import box, mapping

from geocube.api.core import make_geocube
from geocube.rasterize import rasterize_points_griddata, rasterize_points_radial

In [ ]:
df = pd.read_csv("../data/external/resource_2021_03_29_2021_04_05.csv")
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'],df['lat'], crs="EPSG:4326"))

gdf = gdf.to_crs("EPSG:28992")

In [ ]:
gdf.total_bounds

#

In [ ]:
provinces = gpd.read_file("../data/external/B1_Provinciegrenzen_van_Nederland/B1_Provinciegrenzen_van_NederlandPolygon.shp")
utrecht = provinces[provinces["PROV_NAAM"] == "Utrecht"]

gdf2 = gpd.sjoin(gdf, utrecht, how="inner", op='within')

len(gdf2)

In [ ]:
len(gdf)

In [ ]:
gdf.total_bounds

In [ ]:
gdf.head()

In [ ]:
gdf = gdf[:1000]

In [ ]:
geo_grid = make_geocube(
    vector_data=gdf,
    measurements=['pm2_5'],
    resolution=(-25, 25),
    rasterize_function=rasterize_points_griddata,
)

geo_grid

In [ ]:
# mask nodata and plot
geo_grid.pm2_5.where(geo_grid.pm2_5!=geo_grid.pm2_5.rio.nodata).plot()

In [ ]:
geo_grid.pm2_5.plot()

## Opening all city .csv's 

In [8]:

path = "../data/raw/city/" # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in tqdm(all_files):
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

100%|██████████| 101/101 [00:14<00:00,  7.10it/s]


In [9]:
# Changing variables to the correct dType

frame["humidity"] = pd.to_numeric(frame["humidity"])
frame["pm2_5"] = pd.to_numeric(frame["pm2_5"])
frame["pressure"] = pd.to_numeric(frame["pressure"])
frame["trip_sequence"] = pd.to_numeric(frame["trip_sequence"])
frame['recording_time'] = pd.to_datetime(frame['recording_time'], format="%Y-%m-%d %H:%M:%S")
frame["date"] = frame['recording_time'].dt.date
frame["year"] = frame['recording_time'].dt.year
frame["month"] = frame['recording_time'].dt.month
frame["week"] = frame['recording_time'].dt.week
frame["day_of_week"] = frame['recording_time'].dt.dayofweek


<ipython-input-9-5cb5ea3259cc>:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  frame["week"] = frame['recording_time'].dt.week


In [11]:
count = Counter([date for date in frame["date"]])

df = pd.DataFrame.from_dict(count, orient='index', columns=["activity"]).reset_index().sort_values(by=['index'])

fig = px.line(df, x='index', y='activity')
fig.show()

In [12]:
frame

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm2_5,pressure,temperature,distance,delta_time,avg_speed_ms,date,year,month,week,day_of_week
0,09a2bc7fefc3539bef267826fe1e6926,11571976,5.123448,52.105698,2019-05-27 13:01:24,5,35,2,1009,23.8,NaN,NaN,NaN,2019-05-27,2019,5,22,0
1,09a2bc7fefc3539bef267826fe1e6926,11571978,5.124464,52.105083,2019-05-27 13:01:46,5,35,2,1009,23.9,97.567513,22.0,4.434887,2019-05-27,2019,5,22,0
2,09a2bc7fefc3539bef267826fe1e6926,11571979,5.124338,52.105125,2019-05-27 13:02:08,5,35,2,1009,23.9,9.809999,22.0,0.445909,2019-05-27,2019,5,22,0
3,09a2bc7fefc3539bef267826fe1e6926,11571981,5.124736,52.104935,2019-05-27 13:02:41,5,34,3,1009,23.9,34.518211,33.0,1.046006,2019-05-27,2019,5,22,0
4,09a2bc7fefc3539bef267826fe1e6926,11572004,5.125147,52.104782,2019-05-27 13:04:09,5,34,2,1009,23.9,32.903768,88.0,0.373906,2019-05-27,2019,5,22,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2838171,f2faa6b4a4f2dccf226b8592ea2462b3,66972321,5.102649,52.119957,2021-05-02 17:56:16,3,41,1,1009,17.3,13.961343,11.0,1.269213,2021-05-02,2021,5,17,6
2838172,f2faa6b4a4f2dccf226b8592ea2462b3,66972323,5.102751,52.119766,2021-05-02 17:56:27,3,41,1,1009,17.3,22.342345,11.0,2.031122,2021-05-02,2021,5,17,6
2838173,f2faa6b4a4f2dccf226b8592ea2462b3,66972325,5.102846,52.119675,2021-05-02 17:56:38,3,41,1,1009,17.3,12.082791,11.0,1.098436,2021-05-02,2021,5,17,6
2838174,f2faa6b4a4f2dccf226b8592ea2462b3,66972324,5.102946,52.119812,2021-05-02 17:56:49,3,41,1,1009,17.3,16.734468,11.0,1.521315,2021-05-02,2021,5,17,6


In [5]:
weeks = frame.groupby(["year","month","week"]).count()

In [6]:
weeks.sort_values(by="pm2_5", ascending=False).head(20)

sensor  air_quality_observed_id    lon    lat  \
year month week                                                  
2019 12    51     78129                    78129  78129  78129   
           49     75192                    75192  75192  75192   
2020 1     4      73504                    73504  73504  73504   
           2      73175                    73175  73175  73175   
2019 11    47     69915                    69915  69915  69915   
     12    50     68076                    68076  68076  68076   
2020 1     3      63155                    63155  63155  63155   
     2     6      61940                    61940  61940  61940   
     3     10     61301                    61301  61301  61301   
2019 11    48     58757                    58757  58757  58757   
           46     57137                    57137  57137  57137   
2020 1     5      57099                    57099  57099  57099   
     3     11     56835                    56835  56835  56835   
2019 11    45     56113                    56113  56113  56113   
2020 2     7      53535                    53535  53535  53535   
           8      43276                    43276  43276  43276   
     5     21     40147                    40147  40147  40147   
2019 10    44     39569                    39569  39569  39569   
2020 6     26     39467                    39467  39467  39467   
           25     37648                    37648  37648  37648   

                 recording_time  trip_sequence  humidity  pm2_5  pressure  \
year month week                                                             
2019 12    51             78129          78129     78129  78129     78129   
           49             75192          75192     75192  75192     75192   
2020 1     4              73504          73504     73504  73504     73504   
           2              73175          73175     73175  73175     73175   
2019 11    47             69915          69915     69915  69915     69915   
     12    50             68076          68076     68076  68076     68076   
2020 1     3              63155          63155     63155  63155     63155   
     2     6              61940          61940     61940  61940     61940   
     3     10             61301          61301     61301  61301     61301   
2019 11    48             58757          58757     58757  58757     58757   
           46             57137          57137     57137  57137     57137   
2020 1     5              57099          57099     57099  57099     57099   
     3     11             56835          56835     56835  56835     56835   
2019 11    45             56113          56113     56113  56113     56113   
2020 2     7              53535          53535     53535  53535     53535   
           8              43276          43276     43276  43276     43276   
     5     21             40147          40147     40147  40147     40147   
2019 10    44             39569          39569     39569  39569     39569   
2020 6     26             39467          39467     39467  39467     39467   
           25             37648          37648     37648  37648     37648   

                 temperature  distance  delta_time  avg_speed_ms  index_right  \
year month week                                                                 
2019 12    51          78129     78129       78129         78129        78129   
           49          75192     75192       75192         75192        75192   
2020 1     4           73504     73504       73504         73504        73504   
           2           73175     73175       73175         73175        73175   
2019 11    47          69915     69915       69915         69915        69915   
     12    50          68076     68076       68076         68076        68076   
2020 1     3           63155     63155       63155         63155        63155   
     2     6           61940     61939       61939         61939        61940   
     3     10          61301     61300       61300         

In [7]:
neighborhoods = gpd.read_file("../data/external/WijkBuurtkaart_2020_v1/buurt_2020_v1.shp")
utrecht_neigh = neighborhoods[['BU_NAAM','GM_NAAM','BEV_DICHTH','AANT_INW', 'AANT_MAN', 'AANT_VROUW','AANTAL_HH','OPP_TOT', 'OPP_LAND', 'OPP_WATER','geometry']][neighborhoods["GM_NAAM"]=="Utrecht"]
utrecht_neigh

,BU_NAAM,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,geometry
3777,"Welgelegen, Den Hommel",Utrecht,4078,1465,705,760,885,39,36,3,"POLYGON ((135006.339 455419.888, 135197.876 45..."
3778,Oog in Al,Utrecht,9859,4345,2165,2175,1535,47,44,3,"POLYGON ((134605.713 455754.201, 134652.927 45..."
3779,Halve Maan-Zuid,Utrecht,8148,1575,830,745,555,23,19,4,"POLYGON ((134072.101 455635.056, 134074.419 45..."
3780,Halve Maan-Noord,Utrecht,9336,1735,830,900,870,25,19,6,"POLYGON ((133770.466 456647.956, 133786.743 45..."
3781,Lombok-Oost,Utrecht,17630,2285,1110,1175,1470,13,13,0,"POLYGON ((135822.953 456037.558, 135715.534 45..."
...,...,...,...,...,...,...,...,...,...,...,...
3883,Veldhuizen,Utrecht,5631,9330,4635,4690,3460,189,166,23,"POLYGON ((129692.900 455386.278, 129786.270 45..."
3884,De Meern-Noord,Utrecht,3925,5735,2875,2860,2080,148,146,2,"POLYGON ((130301.248 456100.621, 130314.591 45..."
3885,De Meern-Zuid,Utrecht,1671,6425,3160,3260,2715,389,385,5,"POLYGON ((130607.044 455121.579, 130987.916 45..."
3886,Bedrijvengebied Oudenrijn,Utrecht,170,265,125,140,115,163,158,5,"POLYGON ((132114.247 455104.252, 132112.341 45..."


In [8]:
utrecht_neigh["measurement_station"] = utrecht_neigh.representative_point()

In [23]:
geo_df = gpd.GeoDataFrame(frame, geometry=gpd.points_from_xy(frame['lon'],frame['lat'], crs="EPSG:4326"))
geo_df = geo_df.to_crs("EPSG:28992")
geo_df = geo_df.drop(['index_right'], axis=1)
geo_df = gpd.sjoin(geo_df, utrecht_neigh, how="inner", op='within')

geo_df

,sensor,air_quality_observed_id,lon,lat,recording_time,trip_sequence,humidity,pm2_5,pressure,temperature,...,GM_NAAM,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,AANTAL_HH,OPP_TOT,OPP_LAND,OPP_WATER,measurement_station
0,09a2bc7fefc3539bef267826fe1e6926,11571976,5.123448,52.105698,2019-05-27 13:01:24,5,35,2,1009,23.8,...,Utrecht,9502,6945,3310,3630,3460,73,73,0,POINT (137067.024 457765.391)
1,09a2bc7fefc3539bef267826fe1e6926,11571978,5.124464,52.105083,2019-05-27 13:01:46,5,35,2,1009,23.9,...,Utrecht,9502,6945,3310,3630,3460,73,73,0,POINT (137067.024 457765.391)
2,09a2bc7fefc3539bef267826fe1e6926,11571979,5.124338,52.105125,2019-05-27 13:02:08,5,35,2,1009,23.9,...,Utrecht,9502,6945,3310,3630,3460,73,73,0,POINT (137067.024 457765.391)
3,09a2bc7fefc3539bef267826fe1e6926,11571981,5.124736,52.104935,2019-05-27 13:02:41,5,34,3,1009,23.9,...,Utrecht,9502,6945,3310,3630,3460,73,73,0,POINT (137067.024 457765.391)
4,09a2bc7fefc3539bef267826fe1e6926,11572004,5.125147,52.104782,2019-05-27 13:04:09,5,34,2,1009,23.9,...,Utrecht,9502,6945,3310,3630,3460,73,73,0,POINT (137067.024 457765.391)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2837482,cd6d213f6c78309cea4e0314896f7790,66818491,5.071970,52.075016,2021-04-30 14:02:22,6,63,14,1013,13.4,...,Utrecht,-99999999,0,0,0,0,67,66,1,POINT (132832.224 453952.313)
2837483,cd6d213f6c78309cea4e0314896f7790,66818493,5.071960,52.075016,2021-04-30 14:02:33,6,63,15,1013,13.4,...,Utrecht,-99999999,0,0,0,0,67,66,1,POINT (132832.224 453952.313)
2837484,cd6d213f6c78309cea4e0314896f7790,66818494,5.071953,52.075012,2021-04-30 14:02:44,6,63,16,1013,13.4,...,Utrecht,-99999999,0,0,0,0,67,66,1,POINT (132832.224 453952.313)
2837485,cd6d213f6c78309cea4e0314896f7790,66818616,5.071949,52.075020,2021-04-30 14:02:56,6,63,17,1013,13.4,...,Utrecht,-99999999,0,0,0,0,67,66,1,POINT (132832.224 453952.313)


In [25]:
weeks_neigh = geo_df.groupby(["year","month","week","day_of_week","BU_NAAM"]).count()

In [27]:
test = weeks_neigh.sort_values(by="pm2_5", ascending=False).head(50).reset_index()

In [29]:
test.groupby(["year","month","week","day_of_week"]).count().sort_values(by="pm2_5", ascending=False).head(50)

BU_NAAM  sensor  air_quality_observed_id  lon  \
year month week day_of_week                                                  
2019 12    51   3                  4       4                        4    4   
2020 1     2    1                  3       3                        3    3   
2019 12    50   3                  2       2                        2    2   
2020 1     4    3                  2       2                        2    2   
                1                  2       2                        2    2   
           3    3                  2       2                        2    2   
           2    0                  2       2                        2    2   
     12    49   6                  2       2                        2    2   
2019 11    47   3                  2       2                        2    2   
           48   0                  1       1                        1    1   
     6     24   3                  1       1                        1    1   
2020 7     29   3                  1       1                        1    1   
           28   2                  1       1                        1    1   
     5     21   0                  1       1                        1    1   
           20   1                  1       1                        1    1   
           19   4                  1       1                        1    1   
     3     10   4                  1       1                        1    1   
     2     8    3                  1       1                        1    1   
           7    5                  1       1                        1    1   
                3                  1       1                        1    1   
           6    3                  1       1                        1    1   
                2                  1       1                        1    1   
2019 6     24   1                  1       1                        1    1   
     10    41   0                  1       1                        1    1   
     11    48   2                  1       1                        1    1   
2020 1     3    2                  1       1                        1    1   
                0                  1       1                        1    1   
           2    2                  1       1                        1    1   
2019 6     24   0                  1       1                        1    1   
     11    45   3                  1       1                        1    1   
           46   4                  1       1                        1    1   
     12    51   2                  1       1                        1    1   
                1                  1       1                        1    1   
                0                  1       1                        1    1   
           50   2                  1       1                        1    1   
           49   4                  1       1                        1    1   
                1                  1       1                        1    1   
     5     22   0                  1       1                        1    1   

                             lat  recording_time  trip_sequence  humidity  \
year month week day_of_week                                                 
2019 12    51   3              4               4              4         4   
2020 1     2    1              3               3              3         3   
2019 12    50   3              2               2              2         2   
2020 1     4    3              2               2              2         2   
                1              2               2              2         2   
           3    3              2               2              2         2   
           2    0              2               2              2         2   
     12    49   6              2               2              2         2   
2019 11    47   3              2               2              2         2   
           48   0              1               1             

In [2]:
import glob
os.chdir("C:/Users/Klant/Documents/GitHub/ADS-Snuffelfiets-Thesis/data/external/city")
result = glob.glob('*.{}'.format("csv"))

In [3]:
result

['resource_2019_05_27_2019_06_03.csv',
 'resource_2019_06_03_2019_06_10.csv',
 'resource_2019_06_10_2019_06_17.csv',
 'resource_2019_06_17_2019_06_24.csv',
 'resource_2019_06_24_2019_07_01.csv',
 'resource_2019_07_01_2019_07_08.csv',
 'resource_2019_07_08_2019_07_15.csv',
 'resource_2019_07_15_2019_07_22.csv',
 'resource_2019_07_22_2019_07_29.csv',
 'resource_2019_07_29_2019_08_05.csv',
 'resource_2019_08_05_2019_08_12.csv',
 'resource_2019_08_12_2019_08_19.csv',
 'resource_2019_08_19_2019_08_26.csv',
 'resource_2019_08_26_2019_09_02.csv',
 'resource_2019_09_02_2019_09_09.csv',
 'resource_2019_09_09_2019_09_16.csv',
 'resource_2019_09_16_2019_09_23.csv',
 'resource_2019_09_23_2019_09_30.csv',
 'resource_2019_09_30_2019_10_07.csv',
 'resource_2019_10_07_2019_10_14.csv',
 'resource_2019_10_14_2019_10_21.csv',
 'resource_2019_10_21_2019_10_28.csv',
 'resource_2019_10_28_2019_11_04.csv',
 'resource_2019_11_04_2019_11_11.csv',
 'resource_2019_11_11_2019_11_18.csv',
 'resource_2019_11_18_201

In [3]:
utrecht = gpd.read_file("../data/external/WijkBuurtkaart_2020_v1/gem_utrecht.shp")
utrecht = utrecht.to_crs("EPSG:28992")

In [4]:
utrecht

,GM_CODE,JRSTATCODE,GM_NAAM,H2O,OAD,STED,BEV_DICHTH,AANT_INW,AANT_MAN,AANT_VROUW,...,P_ANT_ARU,P_SURINAM,P_TURKIJE,P_OVER_NW,OPP_TOT,OPP_LAND,OPP_WATER,Shape_Leng,Shape_Area,geometry
0,GM0344,2020GM0344,Utrecht,NEE,3444,1,3811,357597,175395,182202,...,1,2,4,8,9921,9383,538,59762.036379,9.920732e+07,"POLYGON ((128020.999 456069.697, 128017.276 45..."


In [6]:
utrecht.bounds.round(1)

,minx,miny,maxx,maxy
0,126434.8,448710.0,141834.4,461609.6


In [13]:
utrecht.bounds

,minx,miny,maxx,maxy
0,126434.772,448709.995,141834.432,461609.608


In [14]:
utrecht.envelope

0    POLYGON ((126434.772 448709.995, 141834.432 44...
dtype: geometry

In [18]:
df

,geometry
0,"POLYGON ((126434.772 448709.995, 141834.432 44..."
